In [1]:
#for dev purpose
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = True

In [2]:
# import python packages
import sys
if sys.version_info[1]==12:
    os.add_dll_directory(r'C:/Program Files/Python/312/Lib/site-packages/clidriver/bin')
else:
    pass
import pandas as pd
import datetime as dt
import time
import warnings
from dateutil.relativedelta import relativedelta
from snp_query_box import DsnpHelperFunction, qnxtQueries
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
print(f"User: {USERNAME} Started creating qnxt_call_df in the shared storage")

User: a845514 Started creating qnxt_call_df in the shared storage


In [3]:
# set the starting time
start_time = time.time()

In [22]:
# calculate the dates
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)
first_date_of_month = dt.datetime.today().replace(day=1)
first_date_of_last_month = first_date_of_month - relativedelta(months=1)
call_log_date2 = pd.to_datetime(last_date_of_last_month).date()
print(call_log_date2)
call_log_date1 = pd.to_datetime(last_date_of_last_month + dt.timedelta(days=1) - relativedelta(years = 1)).date()
print(call_log_date1)

reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

2025-03-31
2024-04-01
2025-03-31


In [7]:
# set the storage and output path
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files/2023-12-31
The folder already exist
Output files will sit in the folder above.


In [19]:
#locate data dump folder
if new_pull:
    pull_call_log_nj = qnxtQueries.pull_call_log_nj(call_log_date1=call_log_date1, call_log_date2=call_log_date2)
    pull_call_log_va = qnxtQueries.pull_call_log_va(call_log_date1=call_log_date1, call_log_date2=call_log_date2)
    pull_call_log_nj = pull_call_log_nj.sort_values(["cl_contact_dts","QNXT_MemberID"], ascending = (False,True))
    pull_call_log_va = pull_call_log_va.sort_values(["cl_contact_dts","QNXT_MemberID"], ascending = (False,True))

    nj_call_df = pull_call_log_nj.sort_values('cl_contact_dts', ascending=False).groupby('QNXT_MemberID').head(1)
    va_call_df = pull_call_log_va.sort_values('cl_contact_dts', ascending=False).groupby('QNXT_MemberID').head(1)
    qnxt_call_df = pd.concat([nj_call_df,va_call_df], axis=0, ignore_index=True)
    qnxt_call_df.to_parquet(f"{output_path}/qnxt_call_df.parquet")

else:
    try:
        qnxt_call_df = pd.read_parquet(f"{output_path}/qnxt_call_df.parquet")

    except:
        pull_call_log_nj = qnxtQueries.pull_call_log_nj(call_log_date1=call_log_date1, call_log_date2=call_log_date2)
        pull_call_log_va = qnxtQueries.pull_call_log_va(call_log_date1=call_log_date1, call_log_date2=call_log_date2)
        pull_call_log_nj = pull_call_log_nj.sort_values(["cl_contact_dts","QNXT_MemberID"], ascending = (False,True))
        pull_call_log_va = pull_call_log_va.sort_values(["cl_contact_dts","QNXT_MemberID"], ascending = (False,True))
    
        nj_call_df = pull_call_log_nj.sort_values('cl_contact_dts', ascending=False).groupby('QNXT_MemberID').head(1)
        va_call_df = pull_call_log_va.sort_values('cl_contact_dts', ascending=False).groupby('QNXT_MemberID').head(1)
        qnxt_call_df = pd.concat([nj_call_df,va_call_df], axis=0, ignore_index=True)
        qnxt_call_df.to_parquet(f"{output_path}/qnxt_call_df.parquet")

100%|██████████| 24838/24838 [00:07<00:00, 3364.09it/s]


In [27]:
print(qnxt_call_df.shape)
print(qnxt_call_df.head(3))

(13257, 13)
  src_sys_nm src_file_nm    MedicareID    QNXT_MemberID          fullname   
0     Dynamo    calllogs  2KJ1X56RQ08   PDZM00004821508  ROBINSON, LOUIS   \
1     Dynamo    calllogs  3J99R78HQ59   PDZM00003156098    PENA, AGAPITA    
2     Dynamo    calllogs  6J54Y12WJ71   PDZM00000351437   JOWERS, SHERYL    

  Contract  PBP cl_call_log_rel_type_desc          cl_contact_dts   
0    H6399  001               Member/Self 2023-12-29 15:51:07.603  \
1    H6399  001               Member/Self 2023-12-29 15:28:03.480   
2    H6399  001               Member/Self 2023-12-29 15:27:58.163   

  cl_attempt_status_key_desc cl_inserted_by_user_nm cl_call_log_type_desc   
0            Successful - FU                A829191           CM Activity  \
1            Successful - FU                A829191           CM Activity   
2            Successful - FU                A832980           CM Activity   

   Source  
0  Dynamo  
1  Dynamo  
2  Dynamo  


In [ ]:
# output the process time
print("Process time -- %s seconds" % (time.time() - start_time))